## Check Validity of Soduku Board 

In [115]:
import Inference as inf
def check_validity(line):
    data = np.array([int(j) for j in line]).reshape((9,9))
    inf.conflict_pairs={}
    inf.conflict_set=set()
    return inf.check_valid_board(data)

## Generate Random Non Valid Soduku Data

In [145]:
import random

file = open("notvalid.txt", "w")

n = 81
for i in range(10000):
    line = ''.join(["{}".format(random.randint(1, 9)) for num in range(0, n)])
    valid = check_validity(line)
    print(valid)
    if not valid:
        line = line + ',0\n'
        file.write(line)
file.close()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

## Process Data for Combined Valid and Non Valid Data for training and testing

In [147]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [148]:
def get_data(): 

    data = pd.read_csv('Dataset/Combined.csv')

    feat_raw = data['Puzzle']
    label_raw = data['Valid']

    feat = []
    label = []

    for i in feat_raw:
    
        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)
    
    feat = np.array(feat)
    feat = feat/9
    feat -= .5    
    
    '''for i in label_raw:
    
        x = np.array(label_raw[i])
        label.append(x)   '''
    
    label = np.array(label_raw)
    
    del(feat_raw)
    del(label_raw)    

    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.5, random_state=42,shuffle=True)
    
    return x_train, x_test, y_train, y_test

In [149]:
x_train, x_test, y_train, y_test = get_data()
print(f'{x_train[0]}----{y_train[0]}')

[[[ 0.5       ]
  [-0.38888889]
  [ 0.38888889]
  [ 0.27777778]
  [-0.27777778]
  [-0.38888889]
  [-0.16666667]
  [ 0.38888889]
  [ 0.27777778]]

 [[-0.16666667]
  [-0.05555556]
  [-0.27777778]
  [-0.27777778]
  [-0.16666667]
  [ 0.5       ]
  [ 0.05555556]
  [ 0.16666667]
  [ 0.38888889]]

 [[ 0.27777778]
  [ 0.5       ]
  [ 0.5       ]
  [ 0.05555556]
  [-0.16666667]
  [-0.27777778]
  [ 0.05555556]
  [ 0.27777778]
  [ 0.38888889]]

 [[-0.05555556]
  [ 0.38888889]
  [-0.05555556]
  [ 0.38888889]
  [ 0.05555556]
  [-0.05555556]
  [-0.38888889]
  [-0.38888889]
  [ 0.16666667]]

 [[-0.27777778]
  [-0.38888889]
  [ 0.38888889]
  [ 0.05555556]
  [ 0.16666667]
  [ 0.38888889]
  [ 0.16666667]
  [-0.05555556]
  [-0.27777778]]

 [[-0.16666667]
  [ 0.27777778]
  [ 0.16666667]
  [ 0.16666667]
  [ 0.05555556]
  [-0.38888889]
  [ 0.27777778]
  [ 0.38888889]
  [ 0.38888889]]

 [[ 0.5       ]
  [-0.16666667]
  [ 0.05555556]
  [ 0.5       ]
  [-0.16666667]
  [-0.27777778]
  [-0.38888889]
  [-0.166666

## Create Model for checking if Soduko board configuarion is Valid

In [16]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape

def get_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

Using TensorFlow backend.


In [151]:
model = get_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization_11 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_12 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten_6 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)               

In [152]:
adam = keras.optimizers.adam(lr=.001)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
9902/9902 [==============================] - 15s 1ms/step - loss: 0.4329 - accuracy: 0.8850
Epoch 2/2
9902/9902 [==============================] - 13s 1ms/step - loss: 0.2568 - accuracy: 0.9966 1s - loss:


## Prediction and Confusion Matrix for test set

In [153]:
y_pred=model.predict(x_test)
y_pred =(y_pred>0.5)
print(y_pred)
print(y_test)

[[False]
 [ True]
 [False]
 ...
 [ True]
 [False]
 [False]]
[0 1 0 ... 1 0 0]


In [154]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4944    7]
 [   0 4951]]


In [155]:
model.save('SodokuValidator.h5')

In [14]:
def process(feat_raw):
    
    feat = np.array([int(j) for j in feat_raw]).reshape((1,9,9,1))
    #feat.append(x)
    
    feat = np.array(feat)
    feat = feat/9
    feat -= .5   
    
    return feat

## Insert random ambiguities in valid dataset

In [218]:
test = '641874532239139687837625419163957248982416753475382196328741965519263874746598321'
data = process(test)
#print(data)
print(model.predict(data))

[[0.633241]]


In [9]:
import pandas as pd
import numpy as np
import random

#Inserts ambiguities at random positions
def create_ambiguous():
    data = pd.read_csv('Dataset/Valid.csv')
    feat = []
    random.seed(32)
    feat_raw = data['Puzzle']
    file1 = open("semilexicalras.txt", "w")
    file2 = open("semilexicalraslabel.txt", "w")
    for i in feat_raw:
        x = np.array([int(j) for j in i]).reshape((9,9))
        y = np.copy(x).tolist()
        print(x)
        rowrange = random.randint(0,8)+1
        colrange = random.randint(0,8)+1
        print(rowrange)
        print(colrange)
        newdict = {}
        for i in range(rowrange):
            for j in range(colrange):
                if x[i][j] == 4:
                    x[i][j] = 0
                    y[i][j] = '9_4'
                    newdict[(i,j)] = {9:50,4:50}
                if x[i][j] == 9:
                    x[i][j] = 0
                    y[i][j] = '4_9'
                    newdict[(i,j)] = {9:50,4:50}
                if x[i][j] == 3:
                    x[i][j] = 0
                    y[i][j] = '5_3'
                    newdict[(i,j)] = {3:50,5:50}
                if x[i][j] == 5:
                    x[i][j] = 0
                    y[i][j] = '3_5'
                    newdict[(i,j)] = {3:50,5:50}
        str1= ''
        str2= ''
        for i in range(9):
            for j in range(9):
                str2=str2+str(y[i][j])
                str1=str1+str(x[i][j])
        line1 = str1 + ',1\n'
        file1.write(line1)
        line2 = str2 + '\n'
        file2.write(line2)
        #print(newdict)
        #print(x.tolist())
        #print(y)
        #print(str1)
        #print(str2)
    file1.close()
    file2.close()

In [10]:
create_ambiguous()

[[5 1 9 6 2 8 4 7 3]
 [8 7 3 9 4 1 6 5 2]
 [6 4 2 3 7 5 9 1 8]
 [9 3 4 7 1 6 2 8 5]
 [2 5 6 4 8 3 7 9 1]
 [1 8 7 2 5 9 3 4 6]
 [4 2 8 5 6 7 1 3 9]
 [7 9 5 1 3 2 8 6 4]
 [3 6 1 8 9 4 5 2 7]]
2
4
[[3 9 5 6 7 8 4 2 1]
 [8 4 2 9 5 1 6 7 3]
 [1 6 7 2 3 4 5 9 8]
 [7 5 6 4 9 3 8 1 2]
 [2 8 9 7 1 5 3 4 6]
 [4 3 1 8 6 2 9 5 7]
 [6 2 8 1 4 9 7 3 5]
 [9 7 3 5 2 6 1 8 4]
 [5 1 4 3 8 7 2 6 9]]
3
5
[[3 7 8 9 4 2 5 1 6]
 [4 1 6 3 5 7 9 2 8]
 [5 2 9 8 6 1 3 7 4]
 [9 3 2 6 7 5 4 8 1]
 [8 6 1 4 9 3 2 5 7]
 [7 4 5 2 1 8 6 9 3]
 [6 8 3 7 2 9 1 4 5]
 [1 9 4 5 8 6 7 3 2]
 [2 5 7 1 3 4 8 6 9]]
4
8
[[6 3 9 5 8 7 1 4 2]
 [8 1 5 4 2 3 7 9 6]
 [2 7 4 6 1 9 5 3 8]
 [7 9 2 8 3 5 6 1 4]
 [5 6 1 7 9 4 8 2 3]
 [4 8 3 2 6 1 9 5 7]
 [3 5 6 9 4 8 2 7 1]
 [1 2 7 3 5 6 4 8 9]
 [9 4 8 1 7 2 3 6 5]]
1
1
[[6 2 5 3 8 4 1 7 9]
 [3 9 7 2 5 1 6 8 4]
 [4 8 1 7 9 6 2 3 5]
 [1 4 3 8 6 5 7 9 2]
 [7 6 8 9 4 2 3 5 1]
 [2 5 9 1 3 7 4 6 8]
 [8 1 4 6 7 9 5 2 3]
 [5 3 6 4 2 8 9 1 7]
 [9 7 2 5 1 3 8 4 6]]
2
6
[[4 3 8 9 7 1 5 2 6]
 [6 2 9 3

8
3
[[9 6 1 5 7 4 2 8 3]
 [2 3 8 6 9 1 5 4 7]
 [4 5 7 8 3 2 6 9 1]
 [5 4 9 3 8 7 1 2 6]
 [3 1 2 4 6 5 8 7 9]
 [8 7 6 1 2 9 3 5 4]
 [1 9 3 7 5 8 4 6 2]
 [7 8 4 2 1 6 9 3 5]
 [6 2 5 9 4 3 7 1 8]]
2
8
[[1 5 4 2 3 7 8 9 6]
 [6 2 8 4 9 1 5 7 3]
 [7 3 9 5 8 6 1 2 4]
 [3 9 7 1 6 2 4 8 5]
 [5 1 2 7 4 8 3 6 9]
 [8 4 6 3 5 9 7 1 2]
 [9 6 3 8 7 4 2 5 1]
 [2 8 5 6 1 3 9 4 7]
 [4 7 1 9 2 5 6 3 8]]
4
5
[[5 8 1 4 3 6 7 2 9]
 [2 7 4 1 8 9 5 3 6]
 [3 6 9 2 5 7 8 4 1]
 [6 1 8 3 4 5 9 7 2]
 [4 2 7 6 9 1 3 8 5]
 [9 5 3 7 2 8 1 6 4]
 [7 9 5 8 6 2 4 1 3]
 [8 3 2 5 1 4 6 9 7]
 [1 4 6 9 7 3 2 5 8]]
5
9
[[1 7 8 2 3 4 9 5 6]
 [2 6 4 5 7 9 3 1 8]
 [3 9 5 6 8 1 7 4 2]
 [9 3 2 4 1 6 8 7 5]
 [8 4 7 3 2 5 1 6 9]
 [5 1 6 7 9 8 4 2 3]
 [6 5 3 9 4 7 2 8 1]
 [4 2 1 8 6 3 5 9 7]
 [7 8 9 1 5 2 6 3 4]]
3
1
[[2 1 8 3 6 4 5 9 7]
 [9 7 5 8 1 2 3 6 4]
 [4 6 3 7 9 5 8 2 1]
 [3 5 1 2 8 6 7 4 9]
 [7 8 9 5 4 1 6 3 2]
 [6 2 4 9 3 7 1 5 8]
 [8 4 2 1 5 3 9 7 6]
 [1 3 7 6 2 9 4 8 5]
 [5 9 6 4 7 8 2 1 3]]
4
8
[[1 6 3 4 8 9 7 2 5]
 [4 7

 [1 6 9 7 2 3 5 4 8]]
2
4
[[6 3 4 9 1 5 7 8 2]
 [2 1 7 3 8 6 9 4 5]
 [9 8 5 4 7 2 1 3 6]
 [8 4 1 5 9 3 2 6 7]
 [7 6 9 2 4 8 5 1 3]
 [5 2 3 1 6 7 4 9 8]
 [3 9 8 7 5 4 6 2 1]
 [1 7 6 8 2 9 3 5 4]
 [4 5 2 6 3 1 8 7 9]]
8
3
[[5 9 2 8 6 7 1 4 3]
 [6 3 1 4 2 5 8 7 9]
 [8 7 4 1 3 9 6 2 5]
 [4 6 3 9 8 1 7 5 2]
 [2 5 8 7 4 3 9 6 1]
 [9 1 7 6 5 2 3 8 4]
 [7 2 5 3 1 6 4 9 8]
 [3 8 9 5 7 4 2 1 6]
 [1 4 6 2 9 8 5 3 7]]
2
1
[[9 6 8 1 2 4 5 3 7]
 [7 4 5 8 6 3 2 1 9]
 [2 3 1 7 9 5 4 6 8]
 [4 7 6 3 1 9 8 2 5]
 [5 1 2 6 8 7 9 4 3]
 [8 9 3 4 5 2 1 7 6]
 [3 8 7 9 4 1 6 5 2]
 [6 2 4 5 7 8 3 9 1]
 [1 5 9 2 3 6 7 8 4]]
9
2
[[9 4 5 8 2 6 7 3 1]
 [7 1 8 3 9 4 5 2 6]
 [3 6 2 7 1 5 9 8 4]
 [1 5 9 2 4 3 6 7 8]
 [6 8 4 5 7 1 3 9 2]
 [2 7 3 6 8 9 1 4 5]
 [5 2 6 4 3 7 8 1 9]
 [8 9 7 1 6 2 4 5 3]
 [4 3 1 9 5 8 2 6 7]]
3
9
[[3 2 4 6 8 1 7 9 5]
 [1 7 5 3 4 9 2 8 6]
 [9 6 8 7 5 2 1 4 3]
 [2 4 7 5 6 8 3 1 9]
 [6 1 3 9 7 4 5 2 8]
 [8 5 9 1 2 3 4 6 7]
 [5 9 6 2 1 7 8 3 4]
 [7 8 1 4 3 6 9 5 2]
 [4 3 2 8 9 5 6 7 1]]
5
5
[[3 

 [6 4 9 8 1 5 7 3 2]]
4
9
[[2 7 8 4 6 3 9 1 5]
 [6 9 5 8 1 2 4 3 7]
 [3 1 4 9 7 5 6 8 2]
 [9 4 3 7 5 1 2 6 8]
 [8 5 6 3 2 4 1 7 9]
 [1 2 7 6 8 9 3 5 4]
 [5 3 9 1 4 7 8 2 6]
 [4 6 2 5 3 8 7 9 1]
 [7 8 1 2 9 6 5 4 3]]
1
2
[[6 2 1 3 4 7 9 5 8]
 [3 5 9 1 8 2 7 4 6]
 [4 8 7 9 5 6 2 1 3]
 [7 1 4 2 3 9 8 6 5]
 [9 6 5 4 1 8 3 2 7]
 [8 3 2 7 6 5 1 9 4]
 [2 9 8 5 7 4 6 3 1]
 [5 7 3 6 9 1 4 8 2]
 [1 4 6 8 2 3 5 7 9]]
9
8
[[3 2 9 5 4 8 6 7 1]
 [1 6 7 9 2 3 5 8 4]
 [4 8 5 1 6 7 3 2 9]
 [2 4 8 6 1 9 7 5 3]
 [7 9 6 3 5 2 1 4 8]
 [5 3 1 8 7 4 9 6 2]
 [9 7 3 2 8 5 4 1 6]
 [6 5 2 4 3 1 8 9 7]
 [8 1 4 7 9 6 2 3 5]]
1
3
[[4 5 2 3 9 6 8 7 1]
 [9 1 3 8 4 7 2 5 6]
 [8 6 7 2 1 5 4 3 9]
 [7 4 1 6 8 2 3 9 5]
 [5 8 9 4 7 3 1 6 2]
 [3 2 6 1 5 9 7 4 8]
 [6 3 8 5 2 4 9 1 7]
 [2 7 5 9 3 1 6 8 4]
 [1 9 4 7 6 8 5 2 3]]
3
3
[[6 8 7 4 9 3 1 5 2]
 [2 4 1 5 7 6 8 3 9]
 [9 5 3 1 8 2 7 4 6]
 [1 6 4 2 5 7 3 9 8]
 [3 9 2 8 4 1 5 6 7]
 [8 7 5 6 3 9 2 1 4]
 [4 1 6 3 2 8 9 7 5]
 [5 2 9 7 1 4 6 8 3]
 [7 3 8 9 6 5 4 2 1]]
7
3
[[4 

3
[[6 2 4 3 1 8 5 7 9]
 [5 7 9 4 6 2 8 3 1]
 [8 3 1 5 9 7 4 2 6]
 [7 5 2 9 8 1 3 6 4]
 [9 8 6 7 3 4 1 5 2]
 [1 4 3 2 5 6 7 9 8]
 [2 6 8 1 7 3 9 4 5]
 [3 1 5 6 4 9 2 8 7]
 [4 9 7 8 2 5 6 1 3]]
5
7
[[6 7 2 9 8 5 4 1 3]
 [5 9 3 1 2 4 6 7 8]
 [4 8 1 3 7 6 2 5 9]
 [8 2 5 4 1 9 3 6 7]
 [9 1 6 8 3 7 5 4 2]
 [3 4 7 6 5 2 8 9 1]
 [7 3 8 5 6 1 9 2 4]
 [1 5 9 2 4 8 7 3 6]
 [2 6 4 7 9 3 1 8 5]]
3
3
[[3 4 8 6 5 1 9 2 7]
 [6 1 7 2 3 9 4 8 5]
 [9 2 5 7 8 4 6 1 3]
 [5 7 1 9 4 6 2 3 8]
 [8 3 6 5 7 2 1 4 9]
 [4 9 2 3 1 8 7 5 6]
 [2 8 3 4 9 7 5 6 1]
 [1 6 9 8 2 5 3 7 4]
 [7 5 4 1 6 3 8 9 2]]
5
2
[[6 5 9 8 7 3 4 1 2]
 [1 8 2 6 4 9 5 3 7]
 [7 4 3 5 1 2 9 8 6]
 [8 2 7 3 5 4 6 9 1]
 [3 1 5 2 9 6 7 4 8]
 [4 9 6 7 8 1 3 2 5]
 [9 6 1 4 2 7 8 5 3]
 [5 3 4 1 6 8 2 7 9]
 [2 7 8 9 3 5 1 6 4]]
7
8
[[6 4 1 5 2 7 3 8 9]
 [8 2 3 1 9 6 4 7 5]
 [5 9 7 3 4 8 2 1 6]
 [4 7 6 9 3 1 5 2 8]
 [2 8 9 6 5 4 7 3 1]
 [1 3 5 8 7 2 6 9 4]
 [7 6 8 4 1 3 9 5 2]
 [9 1 2 7 6 5 8 4 3]
 [3 5 4 2 8 9 1 6 7]]
2
6
[[7 4 1 5 9 6 8 3 2]
 [8 6 9

...

7
2
[[4 5 3 8 7 2 6 1 9]
 [9 1 7 4 5 6 3 8 2]
 [6 2 8 3 1 9 7 4 5]
 [3 8 5 2 6 4 1 9 7]
 [2 7 4 9 8 1 5 3 6]
 [1 9 6 7 3 5 4 2 8]
 [5 6 9 1 2 3 8 7 4]
 [8 4 1 5 9 7 2 6 3]
 [7 3 2 6 4 8 9 5 1]]
7
2
[[9 2 5 7 4 1 3 6 8]
 [6 3 8 9 2 5 7 4 1]
 [1 7 4 6 3 8 5 2 9]
 [5 6 7 4 1 2 8 9 3]
 [3 8 9 5 7 6 2 1 4]
 [2 4 1 8 9 3 6 7 5]
 [4 5 3 2 6 9 1 8 7]
 [8 9 2 1 5 7 4 3 6]
 [7 1 6 3 8 4 9 5 2]]
7
9
[[1 3 5 6 8 7 9 2 4]
 [2 8 4 9 1 3 7 5 6]
 [6 9 7 5 4 2 8 1 3]
 [4 2 3 1 9 5 6 8 7]
 [9 6 1 7 2 8 4 3 5]
 [5 7 8 3 6 4 1 9 2]
 [8 5 9 2 7 6 3 4 1]
 [3 1 6 4 5 9 2 7 8]
 [7 4 2 8 3 1 5 6 9]]
5
1
[[1 8 5 7 6 3 4 2 9]
 [4 9 2 8 5 1 7 6 3]
 [7 3 6 2 9 4 1 5 8]
 [6 5 3 4 7 8 2 9 1]
 [2 1 9 6 3 5 8 7 4]
 [8 4 7 1 2 9 6 3 5]
 [9 7 8 3 1 6 5 4 2]
 [5 2 4 9 8 7 3 1 6]
 [3 6 1 5 4 2 9 8 7]]
3
1
[[6 4 2 5 8 1 3 7 9]
 [8 5 7 3 9 4 6 2 1]
 [1 9 3 7 6 2 4 8 5]
 [7 3 6 1 4 9 8 5 2]
 [5 8 1 6 2 7 9 3 4]
 [9 2 4 8 5 3 7 1 6]
 [2 7 5 4 3 6 1 9 8]
 [4 1 9 2 7 8 5 6 3]
 [3 6 8 9 1 5 2 4 7]]
2
5
[[8 6 7 9 2 3 5 4 1]
 [9 4

 [6 1 7 3 9 2 5 4 8]]
2
6
[[5 3 6 4 8 9 2 7 1]
 [1 2 7 6 5 3 8 9 4]
 [4 9 8 1 2 7 3 6 5]
 [6 5 1 7 9 8 4 3 2]
 [2 4 9 3 6 1 5 8 7]
 [7 8 3 2 4 5 9 1 6]
 [8 6 5 9 1 4 7 2 3]
 [9 7 2 5 3 6 1 4 8]
 [3 1 4 8 7 2 6 5 9]]
3
8
[[5 1 8 9 4 2 7 6 3]
 [2 4 6 7 5 3 9 1 8]
 [9 3 7 8 6 1 2 4 5]
 [1 2 4 5 3 8 6 7 9]
 [6 9 3 2 1 7 5 8 4]
 [8 7 5 4 9 6 3 2 1]
 [7 5 9 1 2 4 8 3 6]
 [3 8 1 6 7 5 4 9 2]
 [4 6 2 3 8 9 1 5 7]]
4
5
[[2 9 7 3 4 5 8 1 6]
 [5 8 1 6 2 9 3 4 7]
 [4 3 6 8 7 1 5 9 2]
 [9 2 8 4 5 6 1 7 3]
 [6 7 5 1 3 8 4 2 9]
 [1 4 3 2 9 7 6 8 5]
 [7 6 9 5 8 4 2 3 1]
 [8 1 2 7 6 3 9 5 4]
 [3 5 4 9 1 2 7 6 8]]
4
7
[[1 7 3 2 6 9 8 5 4]
 [5 8 9 1 4 3 2 7 6]
 [6 2 4 7 5 8 9 3 1]
 [4 5 6 3 7 2 1 9 8]
 [9 1 7 6 8 4 3 2 5]
 [2 3 8 9 1 5 6 4 7]
 [7 9 1 4 2 6 5 8 3]
 [3 4 5 8 9 1 7 6 2]
 [8 6 2 5 3 7 4 1 9]]
3
1
[[9 7 5 4 1 2 3 8 6]
 [3 8 1 9 6 5 2 7 4]
 [2 6 4 3 8 7 1 9 5]
 [6 1 3 2 7 8 5 4 9]
 [5 4 7 6 9 1 8 2 3]
 [8 2 9 5 4 3 6 1 7]
 [4 5 8 7 2 6 9 3 1]
 [1 9 6 8 3 4 7 5 2]
 [7 3 2 1 5 9 4 6 8]]
7
3
[[4 

8
[[7 6 8 4 2 5 1 3 9]
 [5 3 1 9 7 8 2 4 6]
 [9 2 4 3 1 6 7 5 8]
 [6 4 7 2 3 9 8 1 5]
 [8 9 5 1 6 4 3 7 2]
 [3 1 2 8 5 7 9 6 4]
 [1 5 6 7 9 2 4 8 3]
 [4 7 9 5 8 3 6 2 1]
 [2 8 3 6 4 1 5 9 7]]
1
8
[[2 9 8 5 1 4 3 7 6]
 [6 7 5 2 3 9 1 4 8]
 [3 1 4 8 6 7 5 9 2]
 [8 4 7 3 9 2 6 5 1]
 [1 2 6 7 8 5 4 3 9]
 [5 3 9 1 4 6 2 8 7]
 [7 8 2 6 5 3 9 1 4]
 [4 6 3 9 7 1 8 2 5]
 [9 5 1 4 2 8 7 6 3]]
8
6
[[4 3 6 8 9 2 5 7 1]
 [5 2 1 4 7 3 6 8 9]
 [9 7 8 5 1 6 4 3 2]
 [2 6 5 1 4 7 8 9 3]
 [3 8 7 2 6 9 1 4 5]
 [1 9 4 3 5 8 2 6 7]
 [6 1 9 7 8 5 3 2 4]
 [8 4 3 9 2 1 7 5 6]
 [7 5 2 6 3 4 9 1 8]]
3
4
[[5 8 2 4 6 7 3 1 9]
 [7 9 1 3 8 2 6 5 4]
 [4 6 3 9 1 5 2 7 8]
 [2 5 4 8 7 6 1 9 3]
 [8 1 7 5 3 9 4 2 6]
 [9 3 6 2 4 1 5 8 7]
 [3 2 5 6 9 8 7 4 1]
 [6 7 8 1 5 4 9 3 2]
 [1 4 9 7 2 3 8 6 5]]
1
6
[[8 1 9 2 5 3 7 6 4]
 [6 3 7 1 4 9 5 8 2]
 [4 5 2 7 8 6 3 1 9]
 [2 6 8 3 7 5 4 9 1]
 [7 9 5 4 2 1 6 3 8]
 [3 4 1 6 9 8 2 5 7]
 [5 7 6 9 1 4 8 2 3]
 [9 2 3 8 6 7 1 4 5]
 [1 8 4 5 3 2 9 7 6]]
9
1
[[8 9 5 4 6 1 3 2 7]
 [3 4 2

[[7 5 9 3 6 4 2 8 1]
 [4 8 1 5 2 7 3 9 6]
 [2 3 6 9 8 1 5 7 4]
 [3 9 4 8 1 5 7 6 2]
 [5 7 2 6 4 3 8 1 9]
 [1 6 8 7 9 2 4 5 3]
 [6 2 5 4 7 9 1 3 8]
 [8 1 7 2 3 6 9 4 5]
 [9 4 3 1 5 8 6 2 7]]
5
9
[[7 8 3 4 5 9 1 6 2]
 [1 2 6 3 7 8 5 4 9]
 [9 4 5 1 6 2 7 8 3]
 [4 6 8 9 3 1 2 7 5]
 [3 7 2 5 8 4 9 1 6]
 [5 1 9 6 2 7 8 3 4]
 [8 9 4 2 1 6 3 5 7]
 [2 5 7 8 4 3 6 9 1]
 [6 3 1 7 9 5 4 2 8]]
2
2
[[8 7 4 2 1 5 6 9 3]
 [3 2 9 7 4 6 5 8 1]
 [6 5 1 9 8 3 7 2 4]
 [4 1 2 6 9 7 3 5 8]
 [7 8 5 3 2 4 9 1 6]
 [9 3 6 8 5 1 2 4 7]
 [1 4 7 5 3 9 8 6 2]
 [2 9 3 1 6 8 4 7 5]
 [5 6 8 4 7 2 1 3 9]]
6
6
[[8 5 7 3 9 6 1 2 4]
 [6 2 4 7 1 5 8 3 9]
 [1 9 3 2 4 8 7 6 5]
 [7 3 6 1 5 9 4 8 2]
 [4 8 2 6 3 7 9 5 1]
 [5 1 9 8 2 4 6 7 3]
 [9 4 8 5 7 3 2 1 6]
 [2 7 5 9 6 1 3 4 8]
 [3 6 1 4 8 2 5 9 7]]
9
5
[[7 5 6 2 8 3 1 9 4]
 [1 4 9 6 5 7 2 8 3]
 [3 2 8 9 1 4 6 5 7]
 [8 6 1 7 3 5 4 2 9]
 [9 7 4 1 6 2 5 3 8]
 [2 3 5 4 9 8 7 1 6]
 [5 1 7 8 4 9 3 6 2]
 [4 8 3 5 2 6 9 7 1]
 [6 9 2 3 7 1 8 4 5]]
8
2
[[8 2 3 7 6 9 1 5 4]
 [4 1 9 3

 [8 7 4 5 1 2 9 6 3]]
7
4
[[8 3 7 9 6 1 2 5 4]
 [2 4 9 5 7 3 1 8 6]
 [6 5 1 4 8 2 3 9 7]
 [9 7 3 2 1 5 4 6 8]
 [4 1 8 6 3 7 9 2 5]
 [5 2 6 8 9 4 7 3 1]
 [3 8 4 1 5 9 6 7 2]
 [7 6 2 3 4 8 5 1 9]
 [1 9 5 7 2 6 8 4 3]]
4
7
[[2 6 5 4 3 8 1 7 9]
 [1 3 9 7 6 2 8 4 5]
 [8 7 4 1 9 5 6 2 3]
 [9 5 7 8 4 6 2 3 1]
 [4 2 3 5 1 7 9 6 8]
 [6 1 8 9 2 3 4 5 7]
 [5 4 2 3 8 1 7 9 6]
 [7 9 1 6 5 4 3 8 2]
 [3 8 6 2 7 9 5 1 4]]
8
7
[[6 7 4 9 3 5 2 1 8]
 [5 1 8 4 2 6 7 9 3]
 [9 3 2 1 8 7 5 4 6]
 [8 5 7 6 4 2 9 3 1]
 [1 2 3 8 7 9 6 5 4]
 [4 9 6 3 5 1 8 2 7]
 [2 8 1 7 9 4 3 6 5]
 [3 6 9 5 1 8 4 7 2]
 [7 4 5 2 6 3 1 8 9]]
3
4
[[1 7 6 9 4 5 3 8 2]
 [8 4 3 2 6 7 5 1 9]
 [2 5 9 3 1 8 4 7 6]
 [7 9 2 4 3 6 1 5 8]
 [5 8 1 7 9 2 6 3 4]
 [6 3 4 8 5 1 2 9 7]
 [9 2 5 6 7 3 8 4 1]
 [4 1 8 5 2 9 7 6 3]
 [3 6 7 1 8 4 9 2 5]]
8
9
[[4 7 8 9 6 2 1 3 5]
 [6 9 3 7 1 5 2 8 4]
 [2 1 5 3 4 8 7 9 6]
 [1 5 6 8 9 7 4 2 3]
 [3 4 9 2 5 6 8 7 1]
 [8 2 7 1 3 4 6 5 9]
 [5 8 4 6 2 3 9 1 7]
 [7 6 1 5 8 9 3 4 2]
 [9 3 2 4 7 1 5 6 8]]
2
2
[[5 

## Run end to end CNN model for semi-lexical tokens

In [23]:
fileSL = open("semilexicalras.txt", "r")
model = keras.models.load_model('SodokuValidator.h5')
sl_test = []
for line in fileSL:
    sl_test.append(line.split(',')[0])
feat = []
for i in sl_test:
    
    x = np.array([int(j) for j in i]).reshape((9,9,1))
    feat.append(x)
    
feat = np.array(feat)
feat = feat/9
feat -= .5

y_pred=model.predict(feat)
y_pred =(y_pred>0.5)
print(sum(y_pred))

[1030]


## Run Semi-Lexical Framework for board with semi-lexical tokens

In [99]:
fileSL = open("semilexicalras.txt", "r")
fileSLLaber = open("semilexicalraslabel.txt", "r")
sl_test = []
sl_label = []

for line in fileSL:
    sl_test.append(line.split(',')[0])
for line in fileSLLaber:
    boardlist1 = []
    boardlist2 = []
    for element in line:
        boardlist1.append(element)
    x = 0
    while (x<len(boardlist1)):
        if x < len(boardlist1)-1 and boardlist1[x+1] == '_':
            strx = boardlist1[x]+boardlist1[x+1]+boardlist1[x+2]
            boardlist2.append(strx)
            x  = x+3
        else:
            boardlist2.append(boardlist1[x])
            x=x+1
        
    sl_label.append(boardlist2)
print(len(boardlist1))
feat = []
for i in sl_test:  
    x = np.array([int(j) for j in i]).reshape((9,9))
    feat.append(x)
print(len(sl_label[0]))
print(sl_label[0])
print(feat[0])


84
82
['3_5', '1', '4_9', '6', '2', '8', '4', '7', '3', '8', '7', '5_3', '4_9', '4', '1', '6', '5', '2', '6', '4', '2', '3', '7', '5', '9', '1', '8', '9', '3', '4', '7', '1', '6', '2', '8', '5', '2', '5', '6', '4', '8', '3', '7', '9', '1', '1', '8', '7', '2', '5', '9', '3', '4', '6', '4', '2', '8', '5', '6', '7', '1', '3', '9', '7', '9', '5', '1', '3', '2', '8', '6', '4', '3', '6', '1', '8', '9', '4', '5', '2', '7', '\n']
[[0 1 0 6 2 8 4 7 3]
 [8 7 0 0 4 1 6 5 2]
 [6 4 2 3 7 5 9 1 8]
 [9 3 4 7 1 6 2 8 5]
 [2 5 6 4 8 3 7 9 1]
 [1 8 7 2 5 9 3 4 6]
 [4 2 8 5 6 7 1 3 9]
 [7 9 5 1 3 2 8 6 4]
 [3 6 1 8 9 4 5 2 7]]


In [117]:
semilexicalcell_predlist = []
count = 0
while count < len(feat):
    semilexicalpreddict = {}
    for i in range(9):
        for j in range(9):
            if feat[count][i][j] == 0:
                mapdict = {}
                predlist = sl_label[count][((i*9)+j)].split('_')
                mapdict[predlist[0]] = 50
                mapdict[predlist[1]] = 50
                semilexicalpreddict[(i,j)] = mapdict
    count = count + 1
    semilexicalcell_predlist.append(semilexicalpreddict)
print(feat[0])
#print(semilexicalcell_predlist[0])

[[3 1 4 6 2 8 4 7 3]
 [8 7 5 4 4 1 6 5 2]
 [6 4 2 3 7 5 9 1 8]
 [9 3 4 7 1 6 2 8 5]
 [2 5 6 4 8 3 7 9 1]
 [1 8 7 2 5 9 3 4 6]
 [4 2 8 5 6 7 1 3 9]
 [7 9 5 1 3 2 8 6 4]
 [3 6 1 8 9 4 5 2 7]]


In [116]:
import Inference as inf
number_count = 0
for i in range(len(feat)):
    val = inf.sd.call_solve_sudoku(feat[i],semilexicalcell_predlist[i])
    if(isinstance(val,np.ndarray)):
        number_count = number_count+1
print(number_count)

9804
